# 0. Import Libraries

In [1]:
suppressPackageStartupMessages(library('Rsamtools'))
suppressPackageStartupMessages(library('GenomicFeatures'))
suppressPackageStartupMessages(library('GenomicAlignments'))
suppressPackageStartupMessages(library('BiocParallel'))
suppressPackageStartupMessages(library('SummarizedExperiment'))
suppressPackageStartupMessages(library('SummarizedExperiment'))
suppressPackageStartupMessages(library('GenomicFeatures'))
suppressPackageStartupMessages(library('DESeq2'))

# 1. Load BAM files

In [24]:
RAW_CSV <- '/home/parkjoonyoung/share/ChIP-exo_optimization/RPPM/0.GitHub/raw_csv_files/raw_DEseq2_HNS.csv'
RPPM_Folder <- "ChIP-mini_HNS"
REF_DIR <- '/Invivo_DEseq2_common_peaks_ref.gff'
TEST_NAME <- "Invitro_Invivo_HNS"
CON_NAME <- 'Invitro'
EXP_NAME <- 'Invivo'
QVAL <- 0.05
LFC <- 1.0

In [25]:
sampleTable <- read.csv(RAW_CSV, row.names=1)
head(sampleTable)

,sample_id,BAM,Output
,<fct>,<fct>,<fct>
0,Invitro_1,/home/parkjoonyoung/share/ChIP-exo_optimization/RPPM/0.GitHub/raw_files/bam/ChIP-mini_M-invitro-Sal-HNS-1.bam,/media/sf_Share/ChIP-exo_optimization/RPPM/0.GitHub/output/
1,Invitro_2,/home/parkjoonyoung/share/ChIP-exo_optimization/RPPM/0.GitHub/raw_files/bam/ChIP-mini_M-invitro-Sal-HNS-2.bam,/media/sf_Share/ChIP-exo_optimization/RPPM/0.GitHub/output/
2,Invivo_1,/home/parkjoonyoung/share/ChIP-exo_optimization/RPPM/0.GitHub/raw_files/bam/ChIP-mini_M-invivo-Sal-HNS-1.bam,/media/sf_Share/ChIP-exo_optimization/RPPM/0.GitHub/output/
3,Invivo_2,/home/parkjoonyoung/share/ChIP-exo_optimization/RPPM/0.GitHub/raw_files/bam/ChIP-mini_M-invivo-Sal-HNS-2.bam,/media/sf_Share/ChIP-exo_optimization/RPPM/0.GitHub/output/


Put filenames into a character vector and check that they all exist.

In [26]:
filenames <- as.character(sampleTable$BAM)
output <- as.character(sampleTable$Output)[1]
folder_dir <-paste(output,RPPM_Folder,sep="")
result_dir <- paste(folder_dir,'/DEseq2/',sep="")
all(file.exists(filenames))

[1] TRUE

The BamFileList function prepares the BAM files to be processed. The `yieldSize` argument states how many reads can be processed at once (default 2,000,000). This can be increased to speed alignment time, or decreased to reduce memory load.

In [27]:
(bamfiles <- BamFileList(filenames, yieldSize=2000000))

BamFileList of length 4
names(4): ChIP-mini_M-invitro-Sal-HNS-1.bam ...

# 2. Load GFF File

`makeTxDbFromGFF` loads the GFF file into a database.  
`exonsBy` extracts the exons from the GFF file.

In [28]:
gff <- paste(folder_dir, REF_DIR, sep="")
txdb <- makeTxDbFromGFF(gff, format="gtf")
exons <- exonsBy(txdb, by="gene")

Import genomic features from the file as a GRanges object ... OK
Prepare the 'metadata' data frame ... OK
Make the TxDb object ... OK


In [29]:
txdb

TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: /media/sf_Share/ChIP-exo_optimization/RPPM/0.GitHub/output/ChIP-mini_HNS/Invivo_DEseq2_common_peaks_ref.gff
# Organism: NA
# Taxonomy ID: NA
# miRBase build ID: NA
# Genome: NA
# transcript_nrow: 642
# exon_nrow: 642
# cds_nrow: 0
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2024-01-11 14:59:08 +0900 (Thu, 11 Jan 2024)
# GenomicFeatures version at creation time: 1.34.8
# RSQLite version at creation time: 2.1.1
# DBSCHEMAVERSION: 1.2

# 3. Count Reads

`summarizeOverlaps` counts the number of reads that overlap each gene in the GFF file. First, we intialize the multiprocessing, using the `workers` argument to set the number of cores to use. The `summarizeOverlaps` arguments are as follows:
* `features`: The genomic features loaded in the previous code block
* `reads`: The bam files listed above
* `mode`: How to deal with potential overlaps. See [HTSeq-count](http://www-huber.embl.de/HTSeq/doc/count.html) documentation.
* `singleEnd`: TRUE if single-end, FALSE if paired-end
* `ignore.strand`: Whether the strand information is useful for mapping, based on library preparation method
    * TRUE: Standard Illumina
    * FALSE: Directional Illumina (Ligation), Standard SOLiD, dUTP, NSR, NNSR
* `preprocess.reads` (optional): Modify reads before aligning
    * invertStrand: Necessary for dUTP, NSR and NNSR library preparation methods
* `fragments`: Whether to count unpaired reads

In [30]:
register(MulticoreParam(workers = 8))
se <- summarizeOverlaps(features=exons, reads=bamfiles,
                        mode="IntersectionNotEmpty",
                        singleEnd=TRUE,
                        ignore.strand=TRUE,
                        fragments=FALSE)

Error in (function (x)  : 함수가 아닌것에 적용하려고 합니다


The final counts are stored in the [SummarizedExperiment](https://www.bioconductor.org/help/workflows/rnaseqGene/#summarizedexperiment) object. 

In [31]:
se

class: RangedSummarizedExperiment 
dim: 642 4 
metadata(0):
assays(1): counts
rownames(642): p0001 p0002 ... p0641 p0642
rowData names(0):
colnames(4): ChIP-mini_M-invitro-Sal-HNS-1.bam
  ChIP-mini_M-invitro-Sal-HNS-2.bam ChIP-mini_M-invivo-Sal-HNS-1.bam
  ChIP-mini_M-invivo-Sal-HNS-2.bam
colData names(0):

Let's add the metadata information into colData, and set the colnames

In [32]:
metadata <- sampleTable
colData(se) <- DataFrame(metadata)
colnames(se) <- colData(se)$sample_id

To view raw counts, use `assay(se)`

In [33]:
head(assay(se))

,Invitro_1,Invitro_2,Invivo_1,Invivo_2
p0001,12657,13818,16556,16210
p0002,9680,10519,15660,15854
p0003,9422,10467,14025,13882
p0004,25280,27512,33904,34675
p0005,22239,24518,26002,26428
p0006,13829,15208,19207,19284


Finally, save the summarizedExperiment object as a checkpoint

In [34]:
colnames(se)
# Designate groups for comparisons by removing the replicate subscript from colnames
colData(se)$group <- gsub('.{2}$', '', colnames(se))
colData(se)$group

[1] "Invitro_1" "Invitro_2" "Invivo_1"  "Invivo_2"

[1] "Invitro" "Invitro" "Invivo"  "Invivo"

In [35]:
dds <- DESeqDataSet(se, design = ~group)
nrow(dds)
dds <- dds[rowSums(assay(dds)) > 0, ]
nrow(dds)
head(assay(dds))

Warning message in DESeqDataSet(se, design = ~group):
“some variables in design formula are characters, converting to factors”

[1] 642

[1] 642

,Invitro_1,Invitro_2,Invivo_1,Invivo_2
p0001,12657,13818,16556,16210
p0002,9680,10519,15660,15854
p0003,9422,10467,14025,13882
p0004,25280,27512,33904,34675
p0005,22239,24518,26002,26428
p0006,13829,15208,19207,19284


# 4. DEseq2

In [36]:
dds <- DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [37]:
res <- results(dds,contrast = c('group',EXP_NAME,CON_NAME)) 

In [38]:
resTable <- data.frame(res@listData,row.names = res@rownames)

In [39]:
head(resTable)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
p0001,14711.90,0.005224871,0.02954586,0.1768393,8.596346e-01,8.712421e-01
p0002,12689.48,0.339106077,0.02974371,11.4009335,4.136597e-30,1.562174e-29
p0003,11790.06,0.186727264,0.03162814,5.9038335,3.551506e-09,6.080178e-09
p0004,30063.95,0.074827285,0.02082085,3.5938625,3.258119e-04,4.422224e-04
p0005,24755.03,-0.137029946,0.02227130,-6.1527583,7.614683e-10,1.343029e-09
p0006,16707.92,0.104370753,0.02639372,3.9543778,7.673407e-05,1.080335e-04


In [40]:
total_deg_dir <- paste(result_dir, TEST_NAME,'_total.csv',sep="")
write.csv(resTable, total_deg_dir, row.names=T, quote=T)

In [41]:
deg <- resTable[complete.cases(resTable[,c('padj','log2FoldChange')]) & 
                            # Make sure that LogFoldChange and p-adjusted are complete (not null)
                            resTable$padj < QVAL & # Adjusted p-value cutoff of 0.05
                            abs(resTable$log2FoldChange) > LFC,] # abs(log2foldChange) cutoff of 1

In [42]:
deg_dir <- paste(result_dir, TEST_NAME,'_deg.csv',sep="")
write.csv(deg, deg_dir, row.names=T, quote=T)